In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import MiniBatchKMeans
from sklearn.mixture import GaussianMixture
from sklearn.cluster import AgglomerativeClustering
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score
import pandas as pd

In [2]:
data = pd.read_csv("2024pitches.csv")
pd.set_option('display.max_columns', None)
data.head(20)

,Unnamed: 0,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,fielder_2,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length,estimated_slg_using_speedangle,delta_pitcher_run_exp,hyper_speed,home_score_diff,bat_score_diff,home_win_exp,bat_win_exp,age_pit_legacy,age_bat_legacy,age_pit,age_bat,n_thruorder_pitcher,n_priorpa_thisgame_player_at_bat,pitcher_days_since_prev_game,batter_days_since_prev_game,pitcher_days_until_next_game,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle
0,162,KC,2024-10-30,77.5,-1.11,5.65,"Buehler, Walker",657077,621111,strikeout,swinging_strike_blocked,NaN,NaN,NaN,NaN,13.0,Alex Verdugo strikes out swinging.,W,L,R,NYY,LAD,S,2.0,NaN,1,2,2024,1.08,-1.37,-0.21,1.07,NaN,NaN,NaN,2,9,Bot,NaN,NaN,NaN,NaN,NaN,NaN,-0.055693,-112.795187,-0.177357,9.125115,24.929254,-43.904096,3.42,1.55,NaN,NaN,NaN,77.3,2404.0,6.6,775296,669257,518692,666158,571970,669242,621035,571771,605141,53.86,NaN,0.000,0.0,1.0,0.0,0.0,NaN,89,4,Knuckle Curve,6,7,6,7,7,6,6,7,Infield shade,Standard,26.0,-0.044,-0.069,73.8,7.7,NaN,0.069,NaN,-1,-1,0.044,0.044,29,28,30,28,1,4,2.0,1.0,NaN,NaN,5.23,-1.08,1.08,53.2
1,174,KC,2024-10-30,78.7,-1.01,5.73,"Buehler, Walker",657077,621111,NaN,swinging_strike,NaN,NaN,NaN,NaN,14.0,Alex Verdugo strikes out swinging.,W,L,R,NYY,LAD,S,NaN,NaN,1,1,2024,1.05,-1.53,0.28,1.17,NaN,NaN,NaN,2,9,Bot,NaN,NaN,NaN,NaN,NaN,NaN,0.840801,-114.505809,-0.056577,8.894269,26.430826,-45.592304,3.42,1.55,NaN,NaN,NaN,78.0,2650.0,6.4,775296,669257,518692,666158,571970,669242,621035,571771,605141,54.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89,3,Knuckle Curve,6,7,6,7,7,6,6,7,Infield shade,Standard,27.0,0.000,-0.025,73.9,8.0,NaN,0.025,NaN,-1,-1,0.044,0.044,29,28,30,28,1,4,2.0,1.0,NaN,NaN,5.28,-1.05,1.05,54.2
2,178,FC,2024-10-30,93.1,-1.19,5.53,"Buehler, Walker",657077,621111,NaN,swinging_strike,NaN,NaN,NaN,NaN,14.0,Alex Verdugo strikes out swinging.,W,L,R,NYY,LAD,S,NaN,NaN,1,0,2024,0.53,0.73,1.28,2.10,NaN,NaN,NaN,2,9,Bot,NaN,NaN,NaN,NaN,NaN,NaN,5.208110,-135.544075,-4.664347,5.452184,28.588943,-22.336039,3.42,1.55,NaN,NaN,NaN,93.7,2528.0,6.5,775296,669257,518692,666158,571970,669242,621035,571771,605141,53.95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89,2,Cutter,6,7,6,7,7,6,6,7,Infield shade,Standard,176.0,0.000,-0.020,74.2,7.2,NaN,0.020,NaN,-1,-1,0.044,0.044,29,28,30,28,1,4,2.0,1.0,NaN,NaN,1.89,-0.53,0.53,44.8
3,191,KC,2024-10-30,78.5,-1.19,5.70,"Buehler, Walker",657077,621111,NaN,ball,NaN,NaN,NaN,NaN,13.0,Alex Verdugo strikes out swinging.,W,L,R,NYY,LAD,B,NaN,NaN,0,0,2024,1.05,-1.40,-0.09,1.11,NaN,NaN,NaN,2,9,Bot,NaN,NaN,NaN,NaN,NaN,NaN,0.430207,-114.185431,-0.349022,8.932325,25.354360,-44.408593,3.34,1.50,NaN,NaN,NaN,77.7,2516.0,6.2,775296,669257,518692,666158,571970,669242,621035,571771,605141,54.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89,1,Knuckle Curve,6,7,6,7,7,6,6,7,Infield shade,Standard,24.0,0.000,0.015,NaN,NaN,NaN,-0.015,NaN,-1,-1,0.044,0.044,29,28,30,28,1,4,2.0,1.0,NaN,NaN,5.16,-1.05,1.05,51.9
4,196,KC,2024-10-30,77.4,-1.23,5.78,"Buehler, Walker",669224,621111,strikeout,swinging_strike,NaN,NaN,NaN,NaN,7.

In [3]:
data.shape

(757713, 114)

In [4]:
columns = [
    "pitch_type", "release_speed", "release_pos_x", "release_pos_z", "batter", "pitcher", "events", "description", "zone", "des",
    "game_type", "stand", "p_throws", "type", "balls", "strikes", "pfx_x", "pfx_z", "plate_x", "plate_z", "on_3b", "on_2b", "on_1b",
    "hc_x", "hc_y", "sv_id", "vx0", "vy0", "vz0", "ax", "ay", "az", "sz_top", "sz_bot", "hit_distance_sc", "launch_speed",
    "launch_angle", "effective_speed", "release_spin_rate", "release_extension", "game_pk", "release_pos_y",
    "estimated_woba_using_speedangle", "launch_speed_angle", "at_bat_number", "pitch_number", "pitch_name", "home_score",
    "away_score", "spin_axis", "delta_home_win_exp", "delta_run_exp", "bat_speed", "swing_length",
    "estimated_slg_using_speedangle", "delta_pitcher_run_exp", "bat_win_exp", "n_thruorder_pitcher",
    "n_priorpa_thisgame_player_at_bat", "arm_angle"]

pitch_data = data[columns]

In [5]:
pd.set_option('display.max_rows', None)
pitch_data.isnull().sum(axis=0)

pitch_type                           13686
release_speed                        14889
release_pos_x                        14888
release_pos_z                        14888
batter                                   0
pitcher                                  0
events                              560351
description                              0
zone                                 14888
des                                      0
game_type                                0
stand                                    0
p_throws                                 0
type                                     0
balls                                    0
strikes                                  0
pfx_x                                14888
pfx_z                                14888
plate_x                              14888
plate_z                              14888
on_3b                               686878
on_2b                               616465
on_1b                               527641
hc_x       

In [6]:
#pitch_data.to_csv('pitchdata.csv', index=False)

In [7]:
features = ["release_speed", "release_pos_x", "release_pos_z", "release_spin_rate", "pfx_x", 
            "pfx_z", "vx0", "vy0", "vz0", "ax", "ay", "az", "release_extension", "spin_axis"]

pitch_data_filtered = pitch_data.dropna(subset=features).copy()
pitch_data_filtered.loc[:, features] = pitch_data_filtered[features].apply(pd.to_numeric, errors="coerce")

scaler = StandardScaler()
pitch_data_filtered.loc[:, features] = scaler.fit_transform(pitch_data_filtered[features])
pitch_data_filtered = pitch_data_filtered.sample(frac=1/2, random_state=4400) #take sample to speed up clustering

pitch_data_filtered["cluster_label"] = -1

cluster_range = range(2, 9)
clustering_results = []
total_pitches = pitch_data_filtered["pitch_type"].nunique()



In [8]:
for i, pitch in enumerate(pitch_data_filtered["pitch_type"].unique()):
    subset = pitch_data_filtered[pitch_data_filtered["pitch_type"] == pitch].copy()

    if len(subset) < 500:  # Skip pitch types with very few data points
        continue

    print(f"Processing {pitch} ({i+1}/{total_pitches})")  # Track progress

    inertia = []
    silhouette_scores = []
    davies_bouldin_scores = []

    for k in cluster_range:
        kmeans = MiniBatchKMeans(n_clusters=k, random_state=4400, batch_size=1000, n_init=10)
        labels = kmeans.fit_predict(subset[features])

        inertia.append(kmeans.inertia_)
        silhouette_avg = silhouette_score(subset[features], labels)
        silhouette_scores.append(silhouette_avg)
        db_index = davies_bouldin_score(subset[features], labels)
        davies_bouldin_scores.append(db_index)

    # Store results
    for j, k in enumerate(cluster_range):
        clustering_results.append({
            "Pitch Type": pitch,
            "Clusters": k,
            "Inertia": inertia[j],
            "Silhouette Score": silhouette_scores[j],
            "Davies-Bouldin Index": davies_bouldin_scores[j]})

clustering_results_df = pd.DataFrame(clustering_results)

Processing FF (1/17)
Processing CH (2/17)
Processing CU (3/17)
Processing SI (4/17)
Processing FC (5/17)
Processing SL (6/17)
Processing ST (7/17)
Processing KC (8/17)
Processing FS (9/17)
Processing SV (12/17)


In [9]:
clustering_results_df

,Pitch Type,Clusters,Inertia,Silhouette Score,Davies-Bouldin Index
0,FF,2,541028.064963,0.401253,1.053415
1,FF,3,506047.702749,0.315703,1.536459
2,FF,4,434908.369175,0.197075,1.664552
3,FF,5,403503.819303,0.155421,1.860878
4,FF,6,381981.042387,0.140435,1.862559
5,FF,7,365412.475291,0.141496,1.797706
6,FF,8,357059.959781,0.131479,1.907368
7,CH,2,204039.236095,0.465610,0.895628
8,CH,3,184366.388593,0.259943,1.727970
9,CH,4,169715.538946,0.163499,2.053419


In [10]:
manual_cluster_counts = {
    "FF": 3,  # Four-seam
    "SI": 3,  # Sinker
    "CH": 3,  # Changeup
    "CU": 3,  # Curveball
    "FC": 2,  # Cutter
    "ST": 2,  # Sweeper
    "KC": 4,  # Knuckle Curve
    "SL": 2,  # Slider
    "FS": 3,  # Split/Fork
    "SV": 3   # Slurve
}

for pitch in pitch_data_filtered["pitch_type"].unique():
    subset = pitch_data_filtered[pitch_data_filtered["pitch_type"] == pitch].copy()

    if len(subset) < 10:  
        continue
    k = manual_cluster_counts.get(pitch, 3)  

    kmeans = KMeans(n_clusters=k, random_state=4400, n_init=10)
    subset["cluster_label"] = kmeans.fit_predict(subset[features])

    # Merge results back into main dataframe
    pitch_data_filtered.loc[subset.index, "cluster_label"] = subset["cluster_label"]

In [11]:
pitch_data_filtered.head()

,pitch_type,release_speed,release_pos_x,release_pos_z,batter,pitcher,events,description,zone,des,game_type,stand,p_throws,type,balls,strikes,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,hc_x,hc_y,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,release_pos_y,estimated_woba_using_speedangle,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length,estimated_slg_using_speedangle,delta_pitcher_run_exp,bat_win_exp,n_thruorder_pitcher,n_priorpa_thisgame_player_at_bat,arm_angle,cluster_label
557819,FF,0.876060,-0.402642,1.768160,621493,608379,field_out,hit_into_play,4.0,Taylor Ward flies out to center fielder Garret...,R,R,R,X,0,0,-0.512065,1.503754,-0.40,2.28,NaN,NaN,NaN,130.69,47.88,NaN,0.324565,-0.856440,-1.953421,-0.542143,1.000195,1.693502,3.36,1.61,374.0,96.7,25.0,95.0,-0.185110,0.754817,746241,53.67,0.43,5.0,6,1,4-Seam Fastball,0,0,0.415614,-0.010,-0.099,73.0,7.1,0.746,0.099,0.510,1,0,55.6,1
352140,CH,-1.015699,-0.402642,0.799522,672761,680573,NaN,ball,11.0,"Wenceel Pérez grounds out, shortstop Carlos Co...",R,L,R,B,2,2,-1.425656,-0.225370,-1.55,3.44,NaN,NaN,NaN,NaN,NaN,NaN,0.068132,0.993178,1.003259,-1.114535,-1.429299,-0.436038,3.25,1.40,NaN,NaN,NaN,84.4,0.060882,0.973572,745886,53.65,NaN,NaN,1,5,Changeup,0,0,1.158135,0.000,0.096,NaN,NaN,NaN,-0.096,0.500,1,0,38.1,0
53376,FF,1.026731,1.840313,-0.340051,676572,579328,NaN,ball,12.0,Eric Wagaman strikes out swinging.,R,R,L,B,1,2,1.014299,1.163598,1.91,4.23,NaN,694384.0,592273.0,NaN,NaN,NaN,-1.052524,-1.045546,0.913074,1.223853,1.859578,1.161015,3.40,1.61,NaN,NaN,NaN,95.7,0.287528,0.536062,746341,53.76,NaN,NaN,6,4,4-Seam Fastball,0,1,-0.519412,0.000,0.025,NaN,NaN,NaN,-0.025,0.596,1,0,34.4,0
179944,CU,-1.199852,1.375701,1.692188,669127,605483,strikeout,swinging_strike_blocked,13.0,Shea Langeliers strikes out swinging.,R,R,L,S,3,2,-1.358808,-1.911974,-1.39,-0.02,NaN,NaN,NaN,NaN,NaN,NaN,-1.198465,1.198783,-1.156094,-0.889848,-1.001181,-1.622525,3.27,1.52,NaN,NaN,NaN,82.8,0.773985,0.754817,745621,53.70,0.00,NaN,47,7,Curveball,1,1,2.093162,-0.038,-0.299,36.9,7.5,NaN,0.299,0.586,3,2,60.8,0
557655,FF,0.742130,1.023236,-0.112136,677951,660761,NaN,swinging_strike,12.0,Bobby Witt Jr. strikes out swinging.,R,R,L,S,0,1,0.668917,1.617139,0.37,3.72,NaN,672580.0,NaN,NaN,NaN,NaN,-0.918091,-0.747892,0.204692,0.821693,1.595129,1.593237,3.51,1.65,NaN,NaN,NaN,92.5,0.414670,-1.213976,746241,54.56,NaN,NaN,48,2,4-Seam Fastball,0,8,-0.436910,0.000,-0.074,66.5,5.6,NaN,0.074,0.993,1,3,40.6,0


In [12]:
pitch_threshold = 10  # minimum pitches thrown in a season

pitch_data_filtered = pitch_data_filtered.groupby(["pitcher", "pitch_type", "cluster_label"]).size().reset_index(name="count")
pitch_data_filtered = pitch_data_filtered[pitch_data_filtered["count"] > pitch_threshold]

# Find the most common cluster for each pitch type per pitcher
most_common_clusters = pitch_data_filtered.groupby(["pitcher", "pitch_type"])["cluster_label"].agg(lambda x: x.value_counts().idxmax()).reset_index()

pitcher_cluster_df = most_common_clusters.pivot(index="pitcher", columns="pitch_type", values="cluster_label").fillna(-1)
pitcher_cluster_df = pitcher_cluster_df.astype(int)


In [13]:
pitcher_cluster_df.head()

pitch_type,CH,CU,EP,FA,FC,FF,FO,FS,KC,KN,SC,SI,SL,ST,SV
pitcher,,,,,,,,,,,,,,,
434378,0,2,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,1,-1,-1
445276,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,1,1,-1,-1
445926,2,1,-1,-1,1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1
448179,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,-1
450203,0,1,-1,-1,1,2,-1,-1,-1,-1,-1,1,-1,-1,-1


In [14]:
pitcher_cluster_df.to_csv('pitch_clusters.csv')